In [1]:
import json
import csv
import nltk
#nltk.download('wordnet')
#nltk.download('omw-1.4')
from nltk.corpus import wordnet as wn
import pandas as pd

# Assignment 1

In [2]:
hotels_path = 'inputs/yelp_dataset/yelp_hotels.json'

def load_json_line_by_line(path):
    reviews = []
    with open(path, encoding='utf-8') as file:
        for line in file:
            if len(line) > 2:
                # If line ends with a coma, remove it.
                line = line[:-2] if line[-2] == ',' else line[:-1]
                reviews.append(json.loads(line))
    return reviews

def test_load_json_line_by_line():
    paths = [
        'inputs/yelp_dataset/yelp_hotels.json',
        'inputs/yelp_dataset/yelp_restaurants.json',
        'inputs/amazon/Musical_Instruments_5.json'
    ]
    
    for path in paths:
        print('Reading file {}\n'.format(path))
        reviews = load_json_line_by_line(path)
        print('{} reviews loaded\n'.format(len(reviews)))
        print('Example review: {}\n'.format(reviews[0]))

#reviews = load_json_line_by_line(hotels_path)
test_load_json_line_by_line()

Reading file inputs/yelp_dataset/yelp_hotels.json

5034 reviews loaded

Example review: {'reviewerID': 'qLCpuCWCyPb4G2vN-WZz-Q', 'asin': '8ZwO9VuLDWJOXmtAdc7LXQ', 'summary': 'summary', 'reviewText': "Great hotel in Central Phoenix for a stay-cation, but not necessarily a place to stay out of town and without a car. Not much around the area, and unless you're familiar with downtown, I would rather have a guest stay in Old Town Scottsdale, etc. BUT if you do stay here, it's awesome. Great boutique rooms. Awesome pool that's happening in the summer. A GREAT rooftop patio bar, and a very very busy lobby with Gallo Blanco attached. A great place to stay, but have a car!", 'overall': 4.0}

Reading file inputs/yelp_dataset/yelp_restaurants.json

158430 reviews loaded

Example review: {'reviewerID': 'rLtl8ZkDX5vH5nAx9C3q5Q', 'asin': '9yKzy9PApeiPPOUJEtnvkg', 'summary': 'summary', 'reviewText': 'My wife took me here on my birthday for breakfast and it was excellent. The weather was perfect whic

# Assignment 2

In [3]:
aspect_hotels_path = 'inputs/aspects/aspects_hotels.csv'

def load_vocab(path):
    vocab = pd.read_csv(path,names = ["aspect","word"])
    vocab = vocab.groupby('aspect')['word'].apply(list)
    
    return vocab

aspect_hotels = load_vocab(aspect_hotels_path)
print(aspect_hotels)

aspect
amenities                            [amenity, amenities, services]
atmosphere        [atmosphere, atmospheres, ambiance, ambiances,...
bar                              [bar, bars, bartender, bartenders]
bathrooms         [bathroom, bathrooms, bath, baths, bathtub, ba...
bedrooms          [bedroom, bedrooms, bed, beds, pillow, pillows...
booking           [booking, book, reservation, reservations, res...
breakfast         [breakfast, breakfasts, morning, mornings, toa...
building          [building, buildings, architecture, architectu...
checking          [checkin, check-in, check in, check ins, check...
cleanliness       [cleanliness, clean, cleaned, cleaning, dirt, ...
coffee                    [coffee, coffees, cafe, cafes, tea, teas]
cuisine           [cuisine, cuisines, dishe, dishes, food, foods...
dinner            [dinner, dinners, evening meal, evening menu, ...
drinks                    [drink, drinks, beer, beers, wine, wines]
events            [event, events, activit

In [8]:
aspect_hotels.keys()

Index(['amenities', 'atmosphere', 'bar', 'bathrooms', 'bedrooms', 'booking',
       'breakfast', 'building', 'checking', 'cleanliness', 'coffee', 'cuisine',
       'dinner', 'drinks', 'events', 'facilities', 'gym', 'internet',
       'location', 'lunch', 'parking', 'pool', 'price', 'restaurant',
       'restrooms', 'service', 'shopping', 'spa', 'staff', 'temperature',
       'transportation'],
      dtype='object', name='aspect')

In [25]:
def expand_vocab(vocab):
    for a in vocab.keys():
        all_words = {}
        all_words.add(vocab[a])
    
        print(a)
        
expand_vocab(aspect_hotels)
    

amenities
atmosphere
bar
bathrooms
bedrooms
booking
breakfast
building
checking
cleanliness
coffee
cuisine
dinner
drinks
events
facilities
gym
internet
location
lunch
parking
pool
price
restaurant
restrooms
service
shopping
spa
staff
temperature
transportation


In [ ]:
synsets = wn.synsets(words[0])
for s in synsets:
    print(s.name())
    print('\t', s.definition())
    print('\t', s.examples())
    print('\t', s.lemma_names())
    print('\tHyponyms:')
    hyponyms = s.hyponyms()
    for h in hyponyms:
        print('\t\t', h.name(), ' => ', h.lemma_names())
    print('\tHyperyms:')
    hypernyms = s.hypernyms()
    for h in hypernyms:
        print('\t\t', h.name(), ' => ', h.lemma_names())

In [24]:
def get_word_synonims(word):
    syns = []
    for syn in wn.synsets(word):
        for l in syn.lemmas():
            syns.append(l.name())
    return syns
    
word = aspect_hotels[aspect_hotels.keys()[0]][1]

print(get_word_synonims(word))

['comforts', 'creature_comforts', 'amenities', 'conveniences', 'agreeableness', 'amenity']
